In [9]:
import torch
import torch.utils.data as data
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
import torchvision.transforms.v2 as tfs
from tqdm import tqdm
from torchvision.datasets import ImageFolder


import os
import json
from PIL import Image


In [19]:
device = torch.device("mps" if torch.backends.mps.is_available() else "cpu")

class DigitModel(nn.Module):
    def __init__(self):
        super().__init__()
        self.fc1 = nn.Linear(28*28, 128)
        self.fc2 = nn.Linear(128, 64)
        self.fc3 = nn.Linear(64, 32)
        self.fc4 = nn.Linear(32, 10)
        
    def forward(self, x):
        x = F.relu(self.fc1(x))
        x = F.relu(self.fc2(x))
        x = F.relu(self.fc3(x))
        x = self.fc4(x)
        return x


    
            

In [20]:
to_tensor = tfs.PILToTensor()
d= ImageFolder("/Users/macbook/Documents/Документы — gwyndolin/GitHub/dl/MNIST/dataset/train", transform=tfs.Compose([
    tfs.ToTensor(), 
    tfs.Grayscale(),
    tfs.ToDtype(torch.float32, scale=True),
    tfs.Normalize((0.5,), (0.5,)),
    tfs.Lambda(lambda x: x.ravel())
]))
train_data = data.DataLoader(d, batch_size=64, shuffle=True)
model = DigitModel().to(device)
model.train()

lr = 0.001
epochs = 50
optimizer  = optim.Adam(model.parameters(), lr=lr)
loss_func = nn.CrossEntropyLoss()

for epoch in range(epochs):
    train_tqdm = tqdm(train_data, leave=True)
    avg_loss = 0
    for i, (x, y) in enumerate(train_tqdm):
        x, y = x.to(device), y.to(device)
        y_pred = model(x)
        loss = loss_func(y_pred, y)
        

        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
        avg_loss += loss.item()
        train_tqdm.set_description(f"Epoch {epoch+1}/{epochs}, Loss {avg_loss/len(train_data)}")
    



Epoch 2/50, Loss 0.039978800353401504:  17%|█▋        | 157/938 [00:04<00:20, 38.44it/s]


KeyboardInterrupt: 

50 без нормализации, lr=0.001 Adam Accuracy: 0.9778
50 с нормализацией, lr=0.001 Adam Accuracy: 0.974

In [13]:
state_dict = model.state_dict()
torch.save(state_dict, "digit_model_50_!normilize.pth")

In [18]:
test_d= ImageFolder("/Users/macbook/Documents/Документы — gwyndolin/GitHub/dl/MNIST/dataset/test", transform=tfs.Compose([
    tfs.ToTensor(), 
    tfs.Grayscale(),
    tfs.ToDtype(torch.float32, scale=True),
    tfs.Normalize((0.5,), (0.5,)),
    tfs.Lambda(lambda x: x.ravel())
]))
test_data = data.DataLoader(test_d, batch_size=500)

Q = 0

model.eval()
with torch.no_grad():
    for i, (x, y) in enumerate(test_data):
        x, y = x.to(device), y.to(device)
        y_pred = model(x)
        y_pred = torch.argmax(y_pred, dim=1)
        Q+= (y_pred == y).sum().item()

Q = Q/len(test_d)
print(f"Accuracy: {Q}")




Accuracy: 0.974
